# 🔍 Semantic Search & Vector Storage

This notebook demonstrates:
1. **Text Chunking** - Split OCR text into semantic chunks
2. **Embedding Generation** - Create vectors using sentence transformers
3. **Vector Storage** - Store and index in Qdrant
4. **Semantic Search** - Query and retrieve relevant chunks
5. **RAG Integration** - Combine search with LLM for Q&A

## Prerequisites:
```bash
pip install sentence-transformers qdrant-client
# Optional for RAG: pip install openai google-generativeai
```

In [ ]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, project_root)
sys.path.insert(0, os.path.join(project_root, 'backend/ai-service'))

import numpy as np
import json
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass, field, asdict
import uuid
import time

print(f"✅ Project root: {project_root}")

---
## 1️⃣ Text Chunking with Metadata

In [ ]:
@dataclass
class ChunkMetadata:
    """Metadata for a text chunk - links back to source"""
    document_id: str
    page_id: str
    chunk_id: str
    chunk_index: int
    start_char: int
    end_char: int
    bbox: Dict[str, int] = None  # Union of line bboxes
    line_indices: List[int] = field(default_factory=list)
    ocr_confidence: float = 1.0
    language: str = "en"

@dataclass
class TextChunk:
    """A chunk of text with full metadata"""
    text: str
    metadata: ChunkMetadata
    embedding: List[float] = None

class SmartChunker:
    """
    Intelligent text chunking that respects:
    - Sentence boundaries
    - Paragraph boundaries
    - Page boundaries (never cross pages)
    """
    
    def __init__(
        self,
        max_chunk_size: int = 400,  # characters
        min_chunk_size: int = 50,
        overlap: int = 50
    ):
        self.max_chunk_size = max_chunk_size
        self.min_chunk_size = min_chunk_size
        self.overlap = overlap
        
        # Sentence endings
        self.sentence_endings = ['. ', '! ', '? ', '.\n', '!\n', '?\n']
    
    def chunk_text(
        self,
        text: str,
        document_id: str,
        page_id: str,
        ocr_confidence: float = 1.0,
        bbox: Dict[str, int] = None
    ) -> List[TextChunk]:
        """
        Split text into overlapping chunks
        
        Returns list of TextChunk with metadata
        """
        if not text.strip():
            return []
        
        chunks = []
        start = 0
        chunk_index = 0
        
        while start < len(text):
            # Find end position
            end = min(start + self.max_chunk_size, len(text))
            
            # Try to find a sentence boundary
            if end < len(text):
                best_break = end
                
                # Look for sentence ending in last 20% of chunk
                search_start = start + int(self.max_chunk_size * 0.8)
                for ending in self.sentence_endings:
                    pos = text.rfind(ending, search_start, end)
                    if pos > start:
                        best_break = pos + len(ending)
                        break
                
                # Fallback to paragraph break
                if best_break == end:
                    para_break = text.rfind('\n\n', search_start, end)
                    if para_break > start:
                        best_break = para_break + 2
                
                # Last resort: word boundary
                if best_break == end:
                    space_break = text.rfind(' ', search_start, end)
                    if space_break > start:
                        best_break = space_break + 1
                
                end = best_break
            
            # Extract chunk text
            chunk_text = text[start:end].strip()
            
            # Skip if too short
            if len(chunk_text) >= self.min_chunk_size:
                chunk_id = f"{page_id}_c{chunk_index:04d}"
                
                chunks.append(TextChunk(
                    text=chunk_text,
                    metadata=ChunkMetadata(
                        document_id=document_id,
                        page_id=page_id,
                        chunk_id=chunk_id,
                        chunk_index=chunk_index,
                        start_char=start,
                        end_char=end,
                        bbox=bbox,
                        ocr_confidence=ocr_confidence
                    )
                ))
                chunk_index += 1
            
            # Move to next chunk with overlap
            start = end - self.overlap
            if start >= len(text) - self.min_chunk_size:
                break
        
        return chunks

# Initialize
chunker = SmartChunker(max_chunk_size=400, min_chunk_size=50, overlap=50)
print("✅ SmartChunker initialized")

In [ ]:
# Test chunking with sample text
sample_text = """
Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. It focuses on developing algorithms that can access data and use it to learn for themselves.

The learning process begins with observations or data, such as examples, direct experience, or instruction. The goal is to look for patterns in data and make better decisions based on the examples we provide.

There are several types of machine learning:

1. Supervised Learning: The algorithm learns from labeled training data, trying to generalize from these examples to unseen situations.

2. Unsupervised Learning: The algorithm tries to find hidden patterns or intrinsic structures in unlabeled data.

3. Reinforcement Learning: The algorithm learns by interacting with an environment, receiving rewards or penalties for actions taken.

Deep learning is a subset of machine learning that uses neural networks with many layers. These deep neural networks can learn hierarchical representations of data, enabling them to perform complex tasks like image recognition and natural language processing.
"""

doc_id = f"doc_{uuid.uuid4().hex[:8]}"
page_id = f"{doc_id}_p01"

chunks = chunker.chunk_text(
    text=sample_text,
    document_id=doc_id,
    page_id=page_id,
    ocr_confidence=0.95
)

print(f"📦 Created {len(chunks)} chunks from {len(sample_text)} characters")
print(f"\n📋 Chunk Details:")
for chunk in chunks:
    meta = chunk.metadata
    print(f"\n   {meta.chunk_id}:")
    print(f"     Chars: {meta.start_char}-{meta.end_char}")
    print(f"     Length: {len(chunk.text)}")
    print(f"     Text: '{chunk.text[:80]}...'")

---
## 2️⃣ Embedding Generation

In [ ]:
try:
    from sentence_transformers import SentenceTransformer
    SENTENCE_TRANSFORMERS_AVAILABLE = True
    print("✅ sentence-transformers available")
except ImportError:
    SENTENCE_TRANSFORMERS_AVAILABLE = False
    print("⚠️ sentence-transformers not installed")

class EmbeddingGenerator:
    """
    Generate embeddings using sentence transformers
    
    Supports multiple models for different use cases:
    - all-MiniLM-L6-v2: Fast, good quality (384 dimensions)
    - all-mpnet-base-v2: Best quality (768 dimensions)
    - paraphrase-multilingual-MiniLM-L12-v2: Multilingual support
    """
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        self.model_name = model_name
        self.model = None
        self.embedding_dim = None
        
        if SENTENCE_TRANSFORMERS_AVAILABLE:
            self._load_model()
    
    def _load_model(self):
        """Load the sentence transformer model"""
        print(f"   Loading model: {self.model_name}...")
        self.model = SentenceTransformer(self.model_name)
        
        # Get embedding dimension
        test_emb = self.model.encode(["test"])
        self.embedding_dim = len(test_emb[0])
        print(f"   ✅ Model loaded. Embedding dim: {self.embedding_dim}")
    
    def embed_texts(self, texts: List[str]) -> List[List[float]]:
        """
        Generate embeddings for a list of texts
        
        Returns list of embedding vectors
        """
        if not SENTENCE_TRANSFORMERS_AVAILABLE:
            # Return random embeddings for demo
            return [np.random.randn(384).tolist() for _ in texts]
        
        embeddings = self.model.encode(texts, convert_to_numpy=True)
        return embeddings.tolist()
    
    def embed_query(self, query: str) -> List[float]:
        """Generate embedding for a single query"""
        return self.embed_texts([query])[0]
    
    def similarity(self, emb1: List[float], emb2: List[float]) -> float:
        """Compute cosine similarity between two embeddings"""
        a = np.array(emb1)
        b = np.array(emb2)
        return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

# Initialize
embedder = EmbeddingGenerator(model_name="all-MiniLM-L6-v2")

In [ ]:
# Generate embeddings for chunks
texts = [chunk.text for chunk in chunks]

start_time = time.time()
embeddings = embedder.embed_texts(texts)
elapsed = time.time() - start_time

# Attach embeddings to chunks
for chunk, emb in zip(chunks, embeddings):
    chunk.embedding = emb

print(f"✅ Generated {len(embeddings)} embeddings in {elapsed:.2f}s")
print(f"   Dimension: {len(embeddings[0])}")
print(f"   Avg embedding time: {elapsed/len(embeddings)*1000:.1f}ms per chunk")

---
## 3️⃣ Vector Storage (Qdrant)

In [ ]:
try:
    from qdrant_client import QdrantClient
    from qdrant_client.models import Distance, VectorParams, PointStruct, Filter, FieldCondition, MatchValue
    QDRANT_AVAILABLE = True
    print("✅ qdrant-client available")
except ImportError:
    QDRANT_AVAILABLE = False
    print("⚠️ qdrant-client not installed")

class VectorStore:
    """
    Vector storage adapter using Qdrant
    
    Can use:
    - In-memory (for testing)
    - Local file (persistent)
    - Remote Qdrant server
    """
    
    def __init__(
        self,
        collection_name: str = "notes_chunks",
        vector_size: int = 384,
        host: str = None,
        port: int = 6333,
        in_memory: bool = True
    ):
        self.collection_name = collection_name
        self.vector_size = vector_size
        
        if not QDRANT_AVAILABLE:
            self.client = None
            self._in_memory_store = []  # Fallback
            print("   Using in-memory fallback (no Qdrant)")
            return
        
        # Initialize client
        if in_memory or host is None:
            self.client = QdrantClient(":memory:")
            print("   Using in-memory Qdrant")
        else:
            self.client = QdrantClient(host=host, port=port)
            print(f"   Connected to Qdrant at {host}:{port}")
        
        # Create collection
        self._create_collection()
    
    def _create_collection(self):
        """Create collection if not exists"""
        try:
            self.client.create_collection(
                collection_name=self.collection_name,
                vectors_config=VectorParams(
                    size=self.vector_size,
                    distance=Distance.COSINE
                )
            )
            print(f"   Created collection: {self.collection_name}")
        except Exception as e:
            print(f"   Collection exists or error: {e}")
    
    def insert_chunks(self, chunks: List[TextChunk]) -> int:
        """
        Insert chunks with embeddings into vector store
        
        Returns number of inserted points
        """
        if not self.client:
            # Fallback to in-memory
            self._in_memory_store.extend(chunks)
            return len(chunks)
        
        points = []
        for chunk in chunks:
            if not chunk.embedding:
                continue
            
            points.append(PointStruct(
                id=str(uuid.uuid4()),
                vector=chunk.embedding,
                payload={
                    "chunk_id": chunk.metadata.chunk_id,
                    "document_id": chunk.metadata.document_id,
                    "page_id": chunk.metadata.page_id,
                    "chunk_index": chunk.metadata.chunk_index,
                    "text": chunk.text,
                    "start_char": chunk.metadata.start_char,
                    "end_char": chunk.metadata.end_char,
                    "ocr_confidence": chunk.metadata.ocr_confidence
                }
            ))
        
        if points:
            self.client.upsert(
                collection_name=self.collection_name,
                points=points
            )
        
        return len(points)
    
    def search(
        self,
        query_embedding: List[float],
        limit: int = 5,
        document_id: str = None,
        min_score: float = 0.0
    ) -> List[Dict[str, Any]]:
        """
        Search for similar chunks
        
        Returns list of results with score and metadata
        """
        if not self.client:
            # In-memory fallback search
            return self._in_memory_search(query_embedding, limit, document_id)
        
        # Build filter
        query_filter = None
        if document_id:
            query_filter = Filter(
                must=[FieldCondition(
                    key="document_id",
                    match=MatchValue(value=document_id)
                )]
            )
        
        results = self.client.search(
            collection_name=self.collection_name,
            query_vector=query_embedding,
            limit=limit,
            query_filter=query_filter,
            score_threshold=min_score
        )
        
        return [
            {
                "score": hit.score,
                **hit.payload
            }
            for hit in results
        ]
    
    def _in_memory_search(
        self,
        query_embedding: List[float],
        limit: int,
        document_id: str
    ) -> List[Dict[str, Any]]:
        """Fallback in-memory search"""
        from sklearn.metrics.pairwise import cosine_similarity
        
        filtered_chunks = self._in_memory_store
        if document_id:
            filtered_chunks = [c for c in filtered_chunks if c.metadata.document_id == document_id]
        
        if not filtered_chunks:
            return []
        
        # Calculate similarities
        embeddings = [c.embedding for c in filtered_chunks]
        sims = cosine_similarity([query_embedding], embeddings)[0]
        
        # Sort and return top-k
        indices = np.argsort(sims)[::-1][:limit]
        
        return [
            {
                "score": float(sims[idx]),
                "chunk_id": filtered_chunks[idx].metadata.chunk_id,
                "document_id": filtered_chunks[idx].metadata.document_id,
                "page_id": filtered_chunks[idx].metadata.page_id,
                "text": filtered_chunks[idx].text,
                "ocr_confidence": filtered_chunks[idx].metadata.ocr_confidence
            }
            for idx in indices
        ]

# Initialize
vector_store = VectorStore(
    collection_name="notes_demo",
    vector_size=embedder.embedding_dim or 384,
    in_memory=True
)

In [ ]:
# Insert chunks into vector store
inserted = vector_store.insert_chunks(chunks)
print(f"✅ Inserted {inserted} chunks into vector store")

---
## 4️⃣ Semantic Search

In [ ]:
class SemanticSearcher:
    """Semantic search over indexed notes"""
    
    def __init__(
        self,
        embedder: EmbeddingGenerator,
        vector_store: VectorStore
    ):
        self.embedder = embedder
        self.vector_store = vector_store
    
    def search(
        self,
        query: str,
        limit: int = 5,
        document_id: str = None,
        min_score: float = 0.3
    ) -> List[Dict[str, Any]]:
        """
        Search for relevant chunks
        
        Returns list of results with score and context
        """
        # Generate query embedding
        query_emb = self.embedder.embed_query(query)
        
        # Search
        results = self.vector_store.search(
            query_embedding=query_emb,
            limit=limit,
            document_id=document_id,
            min_score=min_score
        )
        
        return results
    
    def search_with_context(
        self,
        query: str,
        limit: int = 5
    ) -> str:
        """
        Search and return formatted context for RAG
        """
        results = self.search(query, limit=limit)
        
        if not results:
            return "No relevant information found."
        
        context_parts = []
        for i, result in enumerate(results):
            context_parts.append(
                f"[Source {i+1}] (Relevance: {result['score']:.2f})\n{result['text']}"
            )
        
        return "\n\n".join(context_parts)

# Initialize
searcher = SemanticSearcher(embedder, vector_store)
print("✅ SemanticSearcher ready")

In [ ]:
# Test semantic search
queries = [
    "What is machine learning?",
    "types of learning algorithms",
    "deep neural networks"
]

for query in queries:
    print(f"\n🔍 Query: '{query}'")
    print("-" * 50)
    
    results = searcher.search(query, limit=3)
    
    for i, result in enumerate(results):
        score_emoji = "🟢" if result['score'] > 0.7 else ("🟡" if result['score'] > 0.5 else "🔴")
        print(f"\n   {i+1}. {score_emoji} Score: {result['score']:.4f}")
        print(f"      Chunk: {result['chunk_id']}")
        print(f"      Text: '{result['text'][:100]}...'")

---
## 5️⃣ RAG (Retrieval Augmented Generation)

In [ ]:
class RAGAnswerer:
    """
    RAG-based Q&A over notes
    
    Uses semantic search + LLM to answer questions
    """
    
    def __init__(self, searcher: SemanticSearcher):
        self.searcher = searcher
        self.llm = None
        self._init_llm()
    
    def _init_llm(self):
        """Initialize LLM (try Gemini, fallback to OpenAI)"""
        # Try Gemini
        try:
            import google.generativeai as genai
            api_key = os.environ.get('GOOGLE_API_KEY')
            if api_key:
                genai.configure(api_key=api_key)
                self.llm = genai.GenerativeModel('gemini-pro')
                print("   ✅ Using Gemini")
                return
        except ImportError:
            pass
        
        # Try OpenAI
        try:
            import openai
            if os.environ.get('OPENAI_API_KEY'):
                self.llm = openai
                print("   ✅ Using OpenAI")
                return
        except ImportError:
            pass
        
        print("   ⚠️ No LLM available - will return context only")
    
    def answer(
        self,
        question: str,
        top_k: int = 3
    ) -> Dict[str, Any]:
        """
        Answer a question using RAG
        
        Returns answer with sources
        """
        # Retrieve context
        results = self.searcher.search(question, limit=top_k)
        context = self.searcher.search_with_context(question, limit=top_k)
        
        if not self.llm:
            return {
                "answer": f"Based on the notes:\n\n{context}",
                "sources": results,
                "model": "none"
            }
        
        # Generate answer with LLM
        prompt = f"""Based on the following notes content, answer the question.
If the answer is not in the notes, say so.

NOTES CONTENT:
{context}

QUESTION: {question}

ANSWER:"""
        
        try:
            if hasattr(self.llm, 'generate_content'):
                # Gemini
                response = self.llm.generate_content(prompt)
                answer = response.text
                model = "gemini-pro"
            else:
                # OpenAI
                response = self.llm.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": prompt}]
                )
                answer = response.choices[0].message.content
                model = "gpt-3.5-turbo"
        except Exception as e:
            answer = f"Error generating answer: {e}\n\nContext:\n{context}"
            model = "error"
        
        return {
            "answer": answer,
            "sources": results,
            "model": model
        }

# Initialize
rag = RAGAnswerer(searcher)

In [ ]:
# Test RAG Q&A
question = "What are the different types of machine learning?"

print(f"❓ Question: {question}")
print("=" * 60)

result = rag.answer(question)

print(f"\n💬 Answer ({result['model']}):")
print(result['answer'])

print(f"\n📚 Sources:")
for source in result['sources']:
    print(f"   - {source['chunk_id']}: score={source['score']:.3f}")

---
## 6️⃣ API Response Format (for Integration)

In [ ]:
def format_search_response(results: List[Dict], query: str) -> Dict[str, Any]:
    """Format search results for API response"""
    return {
        "query": query,
        "total_results": len(results),
        "results": [
            {
                "chunk_id": r["chunk_id"],
                "document_id": r["document_id"],
                "page_id": r["page_id"],
                "score": round(r["score"], 4),
                "text": r["text"],
                "highlight_bbox": r.get("bbox")  # For UI highlighting
            }
            for r in results
        ]
    }

# Demo
results = searcher.search("neural networks")
api_response = format_search_response(results, "neural networks")

print("📡 API Response:")
print(json.dumps(api_response, indent=2))

---
## 🎯 Summary

This notebook demonstrated:

| Component | Purpose | Technology |
|-----------|---------|------------|
| SmartChunker | Semantic text chunking | Custom algorithm |
| EmbeddingGenerator | Vector embeddings | Sentence Transformers |
| VectorStore | Storage & indexing | Qdrant |
| SemanticSearcher | Similarity search | Cosine similarity |
| RAGAnswerer | Q&A with LLM | Gemini/OpenAI |

### Integration Points:
- `SmartChunker` → After OCR text extraction
- `VectorStore` → Qdrant service adapter
- `SemanticSearcher` → `/api/search` endpoint
- `RAGAnswerer` → `/api/notes/ask` endpoint